In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from preprocess import preprocessor
from preprocess import clean_df
from preprocess import define_vocab_to_remove
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer 

In [14]:
data = pd.read_csv("labelled_newscatcher_dataset.csv", sep=";")
data

sample_data = data.sample(n=10000, random_state=42).reset_index(drop=True)
sample_X,sample_y = clean_df(sample_data)

X,y = clean_df(data)
words_to_remove = define_vocab_to_remove(X)

X_train,X_test,y_train,y_test = train_test_split(sample_X,sample_y,test_size=0.2)

for i in range(len(X_train)):
    X_train.iloc[i] = preprocessor(X_train.iloc[i], words_to_remove)

c:\Users\quent\OneDrive\Bureau\NLP\NLP_project\preprocess.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["title"] = filtered_data["title"].astype(str)
c:\Users\quent\OneDrive\Bureau\NLP\NLP_project\preprocess.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["title"] = filtered_data["title"].astype(str)


In [15]:
class_labels = ['SCIENCE', 'TECHNOLOGY', 'HEALTH', 'WORLD', 'ENTERTAINMENT', 'SPORTS', 'BUSINESS', 'NATION']

label_encoder = LabelEncoder() # instantiate a label encoder 
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.fit_transform(y_test)

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import Vocab
from torch.utils.data import DataLoader, TensorDataset


In [25]:
X_train = X_train.reset_index(drop=True)

full_text = ' '.join(X_train)
split_text = full_text.split()
set_split_text = set(split_text)


In [26]:
class TitleClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, nb_classes):
        super(TitleClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim,hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim,nb_classes)

    def forward(self,X):
        embedded = self.embedding(X)
        output , _ = self.rnn(embedded)
        last_hidden = output[:, -1, :]
        logits = self.fc(last_hidden)
        return logits


In [27]:
X_train

0         joe biden kamala harri say trump left us tatter
1            liz hurley son damian spot gal pal wear jean
2               atletico report two posit coronaviru test
3                  smart tv come roku here that bad thing
4       wechat ban could impact appl global iphon ship...
                              ...                        
7995          fall billboard injur two heavi rain karachi
7996    pj polic 19yearold allegedli gang rape seven s...
7997    california megachurch draw thousand inperson s...
7998     pandit jasraj tribut ar rahman shabana azmi star
7999    russian weightlift alexand sedykh break knee 4...
Name: title, Length: 8000, dtype: object

In [32]:
X_train

0         joe biden kamala harri say trump left us tatter
1            liz hurley son damian spot gal pal wear jean
2               atletico report two posit coronaviru test
3                  smart tv come roku here that bad thing
4       wechat ban could impact appl global iphon ship...
                              ...                        
7995          fall billboard injur two heavi rain karachi
7996    pj polic 19yearold allegedli gang rape seven s...
7997    california megachurch draw thousand inperson s...
7998     pandit jasraj tribut ar rahman shabana azmi star
7999    russian weightlift alexand sedykh break knee 4...
Name: title, Length: 8000, dtype: object

In [36]:
vocab_size = len(set_split_text)
embedding_dim = 100
hidden_dim = 64
nb_classes = 8
batch_size = 32
nb_epochs = 5
learning_rate = 0.001

Model = TitleClassifier(vocab_size, embedding_dim, hidden_dim, nb_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Model.parameters(), lr=learning_rate)

X_train_tensor = torch.tensor(X_train.astype(int), dtype=torch.long)
y_enc_tensor = torch.tensor(y_train_enc[:1000], dtype=torch.long)

train_data = TensorDataset(X_train, y_train_enc)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)

for epoch in range(nb_epochs):
    Model.train()
    for text,y in train_data:
        optimizer.zero_grad()
        output = Model(text)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

ValueError: invalid literal for int() with base 10: 'joe biden kamala harri say trump left us tatter'

In [224]:
X_test_tensor = torch.tensor(X_test.toarray(), dtype=torch.long)
test_loader = DataLoader(X_test_tensor, batch_size=batch_size, shuffle=False)

Model.eval()
with torch.no_grad():
    total_test_samples = 0
    correct_predictions = 0
    for text in test_loader:
        text = torch.LongTensor(text)
        outputs = Model(text)

        _, predicted_labels = torch.max(outputs, dim=1)

        total_test_samples += len(text)
        correct_predictions += (predicted_labels == torch.tensor(y_test_enc[:32])).sum().item()

accuracy = correct_predictions / total_test_samples
print(accuracy)

RuntimeError: The size of tensor a (4) must match the size of tensor b (32) at non-singleton dimension 0